# Creación de paquetes ARP

### Descripción: 
Con este código se va a intentar crear diferentes tipos de paquetes ARP, mediante IA generativa. Se pretende pasar una descripción de estos paquetes al modelo y que este sea capaz de crear todos los campos necesarios para replicar un paquete con esas mismas especificaciones.

In [ ]:
import random
import re
import pickle

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether,ARP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de un paquete ICMP

<center><img src="https://dis.um.es/~lopezquesada/documentos/IES_1314/LMSGI/curso/xhtml/xhtml11/imagenes/arp.jpg">
    
<center> Esta imagen es genérica para una conversación ARP.


In [2]:
query = "In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.\n\n"
query += "ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast\n"

query += "This is an ARP specific IP query summary:\n"
query += '75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1\n'
query += "This is the command to generate this packet\n"
query += 'scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0d:09:e2:5d")\n\n'

query += "This is an ARP broadcast query summary:\n"
query += 'c2:3d:19:6c:00:01 Broadcast ARP 64 Who has 192.168.121.2? Tell 192.168.121.253\n'
query += "This is the command to generate this packet\n"
query += 'scapy.Ether(src="c2:3d:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="192.168.121.2", psrc="192.168.121.253", hwdst="00:00:00:00:00:00", hwsrc="c2:3d:19:6c:00:01")\n\n'

query += "This is an ARP response summary:\n"
query += "c2:3c:19:6c:00:01 c2:3d:19:6c:00:01 ARP 60 10.0.0.2 is at c2:3c:19:6c:00:01"
query += "This is the command to generate the packet:\n"
query += 'scapy.Ether(src="c2:3c:19:6c:00:01", dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="c2:3d:19:6c:00:01",  pdst=socket.inet_ntoa(struct.pack(">I", random.randint(1, 0xffffffff))))\n\n'

query += "This is an Gratuitous ARP summary:\n"
query += "c2:3c:19:6c:00:01     Broadcast             ARP      60     Gratuitous ARP for 10.0.0.2"
query += "This is the command to generate the packet:\n"
query += 'scapy.Ether(src="c2:3c:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="FF:FF:FF:FF:FF:FF",  pdst="10.0.0.2")\n\n'

query += 'Take your time (few seconds) to think if the packets and their variables make sense for an ARP packet, if not, correct them\n'

In [3]:
print(query)

In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0d:09:e2:5d")

This is an ARP broadcast query summary:
c2:3d:19:6c:00:01 Broadcast ARP 64 Who has 192.168.121.2? Tell 192.168.121.253
This is the command 

## Carga de posibles descripciones

In [4]:
with open("../../data/text/ARP_TheUltimatePCAP_v2.txt","r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

999

In [5]:
test_group = random.choices(packets_summary,k=200)

### Carga de posibles conversaciones anteriores para concatenarlas

In [6]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [7]:
try:
    responses = pickle.load( open("../../data/Packets/ARP/pickle/ARP_packet_pairs_Aday_validation_v3.pkl", "rb" ))
except:
    responses = []

## Generación de comandos de paquetes conversaciones TCP

In [12]:

index = 0

for i in range(int(len(test_group)/10)):
    
    query1 = query
    
    summaries = test_group[index:index+10]
    index += 10
    #print(index)
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    

    text_sum = ""
    
    query1 += "\nBelow, I'll give you the descriptions of the packets you have to generate, the previous where just examples\n\n"
    
    for summary in summaries:
        query1 += "This is a packet summary:\n"
        query1 += summary + "\n"
        text_sum += summary + "\n"
        
    query1 += "\nGenerate python code for creating JUST the packets of the summaries given with scapy framework and put them all in a list named pkt_list. DO NOT create commands for other types of ARP.\n"
    query1 += "Use the following structure for the pkt_list: pkt_list[scapy.Ether(...)/scapy.ARP(..), scapy.Ether(...)/scapy.ARP(..), ...]"
    
    print(query1)
    
    #Esta es la petición que le mandas al modelo de IA
    completion = openai.completions.create(
        model="gpt-3.5-turbo-instruct", 
        prompt=query1,
        max_tokens=1500,
        echo = False,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    exec(responses[-1].completion.choices[0].text)
    
    with open("../../data/Packets/ARP/pcap/Validation_v3_ARPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Packets/ARP/pickle/ARP_packet_pairs_Aday_validation_v3.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0d:09:e2:5d")

This is an ARP broadcast query su

...Generated!
Number of completitions done: 10
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 4)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0

...Generated!
Number of completitions done: 13
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 7)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0

...Generated!
Number of completitions done: 16
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 10)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:

...Generated!
Number of completitions done: 19
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 13)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:

...Generated!
Number of completitions done: 22
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 16)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:

...Generated!
Number of completitions done: 26
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 20)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:

...Generated!
Number of completitions done: 30
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 24)
In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:

KeyboardInterrupt: 

In [217]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

a4:9b:4f:53:00:0d     a4:9b:4f:53:00:0e     ARP      60     192.168.2.13 is at a4:9b:4f:53:00:0d
a4:9b:4f:53:00:0e     Broadcast             ARP      60     Gratuitous ARP for 192.168.2.14 (Reply)



pkt_list = [
    scapy.Ether(src="a4:9b:4f:53:00:0d", dst="a4:9b:4f:53:00:0e")/scapy.ARP(op=2, psrc="192.168.2.13", hwsrc="a4:9b:4f:53:00:0d", hwdst="a4:9b:4f:53:00:0e"),
    scapy.Ether(src="a4:9b:4f:53:00:0e", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=2, psrc="192.168.2.14", hwsrc="a4:9b:4f:53:00:0e", hwdst="FF:FF:FF:FF:FF:FF", pdst="192.168.2.14")
]


### Generación de paquetes

In [220]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<Ether  dst=a4:9b:4f:53:00:0e src=a4:9b:4f:53:00:0d type=ARP |<ARP  op=is-at hwsrc=a4:9b:4f:53:00:0d psrc=192.168.2.13 hwdst=a4:9b:4f:53:00:0e |>>, <Ether  dst=FF:FF:FF:FF:FF:FF src=a4:9b:4f:53:00:0e type=ARP |<ARP  op=is-at hwsrc=a4:9b:4f:53:00:0e psrc=192.168.2.14 hwdst=FF:FF:FF:FF:FF:FF pdst=192.168.2.14 |>>]


In [222]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [225]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Packets/ARP/pcap/Validation_v2_ARPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)